In [158]:
from yahoo_oauth import OAuth2
import yahoo_fantasy_api as yfa

In [184]:
#connect to yahoo api

sc  = OAuth2(None, None, from_file='keys.json')

[2024-11-06 08:34:01,553 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2024-11-06 08:34:01,555 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 37.535324811935425
[2024-11-06 08:34:01,555 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [160]:
#create game class variable
gameID = yfa.game.Game(sc, "nfl")
gameID.league_ids(year=2024)
players = []
teamKeys = []


In [130]:
#obtain league variable and append minnesota details
lg = yfa.league.League(sc, '449.l.339288')
id = lg.player_details("minnesota")[0]
id = id['player_id']
players.append(id)
players

['100016']

In [131]:
#create teams
teams = lg(sc, '449.l.339288').teams()
teams
for i in teams:
    teamKeys.append(teams[i]['team_key'])


In [134]:
#append players 
teamPlayers = lg.taken_players()
teamPlayers
teamPlayers


[{'player_id': 25785,
  'name': 'Russell Wilson',
  'position_type': 'O',
  'eligible_positions': ['QB'],
  'percent_owned': 16,
  'status': ''},
 {'player_id': 25812,
  'name': 'Kirk Cousins',
  'position_type': 'O',
  'eligible_positions': ['QB'],
  'percent_owned': 74,
  'status': ''},
 {'player_id': 26534,
  'name': 'Justin Tucker',
  'position_type': 'K',
  'eligible_positions': ['K'],
  'percent_owned': 99,
  'status': ''},
 {'player_id': 26650,
  'name': 'DeAndre Hopkins',
  'position_type': 'O',
  'eligible_positions': ['WR', 'W/R/T'],
  'percent_owned': 84,
  'status': ''},
 {'player_id': 26658,
  'name': 'Zach Ertz',
  'position_type': 'O',
  'eligible_positions': ['TE', 'W/R/T'],
  'percent_owned': 38,
  'status': ''},
 {'player_id': 26662,
  'name': 'Geno Smith',
  'position_type': 'O',
  'eligible_positions': ['QB'],
  'percent_owned': 58,
  'status': ''},
 {'player_id': 26686,
  'name': 'Travis Kelce',
  'position_type': 'O',
  'eligible_positions': ['TE', 'W/R/T'],
  'pe

In [135]:
#append first batch of season stats
for i in teamPlayers:
    i['season'] = lg.player_stats(i["player_id"], 'season')
    i['week_1'] = lg.player_stats(i['player_id'], 'week', week=1)
    i['week_2'] = lg.player_stats(i['player_id'], 'week', week=2)

    

In [161]:
#append second batch of season stats
for i in teamPlayers:    
    i['week_3'] = lg.player_stats(i['player_id'], 'week', week=3)
    i['week_4'] = lg.player_stats(i['player_id'], 'week', week=4)


In [163]:
#append third batch
for i in teamPlayers:
    i['week_5'] = lg.player_stats(i['player_id'], 'week', week=5)
    i['week_6'] = lg.player_stats(i['player_id'], 'week', week=6)


In [165]:
#append fourth batch
for i in teamPlayers:
    i['week_7'] = lg.player_stats(i['player_id'], 'week', week=7)
    i['week_8'] = lg.player_stats(i['player_id'], 'week', week=8)

In [167]:
#append fith batch
for i in teamPlayers:
    i['week_9'] = lg.player_stats(i['player_id'], 'week', week=9)

In [181]:

free = lg.free_agents("DT")
free


[]

In [188]:
teamPlayers


[{'player_id': 25785,
  'name': 'Russell Wilson',
  'position_type': 'O',
  'eligible_positions': ['QB'],
  'percent_owned': 16,
  'status': '',
  'season': [{'player_id': 25785,
    'name': 'Russell Wilson',
    'position_type': 'O',
    'Pass Yds': 542.0,
    'Pass TD': 3.0,
    'Int': 0.0,
    'Sack': 5.0,
    'Rush Att': 6.0,
    'Rush Yds': 10.0,
    'Rush TD': 1.0,
    'Rec': 0.0,
    'Rec Yds': 0.0,
    'Rec TD': 0.0,
    'Ret Yds': 0.0,
    'Ret TD': 0.0,
    '2-PT': 0.0,
    'Fum': 1.0,
    'Fum Lost': 1.0,
    'Fum Ret TD': 0.0,
    'Pick Six': 0.0,
    '40 Yd Comp': 2.0,
    '40 Yd Pass TD': 0.0,
    '40 Yd Rush': 0.0,
    '40 Yd Rush TD': 0.0,
    '40 Yd Rec': 0.0,
    '40 Yd Rec TD': 0.0,
    'Targets': 0.0}],
  'week_1': [{'player_id': 25785,
    'name': 'Russell Wilson',
    'position_type': 'O',
    'Pass Yds': 0.0,
    'Pass TD': 0.0,
    'Int': 0.0,
    'Sack': 0.0,
    'Rush Att': 0.0,
    'Rush Yds': 0.0,
    'Rush TD': 0.0,
    'Rec': 0.0,
    'Rec Yds': 0.0,
    '

In [190]:
#save file as a json for safekeeping
import json
with open("nflData.json", 'w') as json_file:
    json.dump(teamPlayers, json_file, indent=4)
print("Data saved to output.json")

Data saved to output.json


In [206]:
#Calculate fantasy football point totals for all offensive class players
for i in teamPlayers:
    if i['position_type'] == 'O':
        season_stats = i["season"][0]  
        i["Fantasy Points"] = (
            season_stats['Pass Yds'] / 25 
            + season_stats['Pass TD'] * 4
            + (-1) * season_stats['Int']
            + (-0.5) * season_stats['Sack']
            + season_stats['Rush Yds'] * 0.1
            + season_stats['Rush TD'] * 6
            + season_stats['Rec']
            + season_stats['Rec Yds'] * 0.1
            + season_stats['Rec TD'] * 6
            + season_stats['Ret Yds'] / 15
            + season_stats['Ret TD'] * 6
            + season_stats['2-PT'] * 2
            + (-0.5) * season_stats['Fum']
            + (-2) * season_stats['Fum Lost']
            + season_stats['Fum Ret TD'] * 6
            + (-4) * season_stats['Pick Six']
            + season_stats["40 Yd Comp"] * 2
            + season_stats["40 Yd Pass TD"] * 3
            + season_stats["40 Yd Rush"] * 2
            + season_stats["40 Yd Rush TD"] * 3
            + season_stats["40 Yd Rec"] * 2
            + season_stats["40 Yd Rec TD"] * 3
            
        )
        i["season"][0]["Fantasy Points"] = (
            season_stats['Pass Yds'] / 25 
            + season_stats['Pass TD'] * 4
            + (-1) * season_stats['Int']
            + (-0.5) * season_stats['Sack']
            + season_stats['Rush Yds'] * 0.1
            + season_stats['Rush TD'] * 6
            + season_stats['Rec']
            + season_stats['Rec Yds'] * 0.1
            + season_stats['Rec TD'] * 6
            + season_stats['Ret Yds'] / 15
            + season_stats['Ret TD'] * 6
            + season_stats['2-PT'] * 2
            + (-0.5) * season_stats['Fum']
            + (-2) * season_stats['Fum Lost']
            + season_stats['Fum Ret TD'] * 6
            + (-4) * season_stats['Pick Six']
            + season_stats["40 Yd Comp"] * 2
            + season_stats["40 Yd Pass TD"] * 3
            + season_stats["40 Yd Rush"] * 2
            + season_stats["40 Yd Rush TD"] * 3
            + season_stats["40 Yd Rec"] * 2
            + season_stats["40 Yd Rec TD"] * 3
        )
        #Calculate fantasy football point totals for all offensive class players for each week
        for x in range(1,10):
            season_stats = i[f"week_{x}"][0]
            i[f"week_{x}"][0]["Fantasy Points"] = (
            season_stats['Pass Yds'] / 25 
            + season_stats['Pass TD'] * 4
            + (-1) * season_stats['Int']
            + (-0.5) * season_stats['Sack']
            + season_stats['Rush Yds'] * 0.1
            + season_stats['Rush TD'] * 6
            + season_stats['Rec']
            + season_stats['Rec Yds'] * 0.1
            + season_stats['Rec TD'] * 6
            + season_stats['Ret Yds'] / 15
            + season_stats['Ret TD'] * 6
            + season_stats['2-PT'] * 2
            + (-0.5) * season_stats['Fum']
            + (-2) * season_stats['Fum Lost']
            + season_stats['Fum Ret TD'] * 6
            + (-4) * season_stats['Pick Six']
            + season_stats["40 Yd Comp"] * 2
            + season_stats["40 Yd Pass TD"] * 3
            + season_stats["40 Yd Rush"] * 2
            + season_stats["40 Yd Rush TD"] * 3
            + season_stats["40 Yd Rec"] * 2
            + season_stats["40 Yd Rec TD"] * 3
            )
    #Calculate fantasy football point totals for all kicker class players
    elif i['position_type'] == 'K':
        season_stats = i["season"][0]  
        i["Fantasy Points"] = (
            season_stats['FG 0-19'] * 3
            + season_stats['FG 20-29'] * 3
            + season_stats['FG 30-39'] * 3
            + season_stats['FG 40-49'] * 4
            + season_stats['FG 50+'] * 5
            + season_stats['PAT Made']
        ) 
        i["season"][0]["Fantasy Points"] = (
            season_stats['FG 0-19'] * 3
            + season_stats['FG 20-29'] * 3
            + season_stats['FG 30-39'] * 3
            + season_stats['FG 40-49'] * 4
            + season_stats['FG 50+'] * 5
            + season_stats['PAT Made']
        )
        #Calculate fantasy football point totals for all kicker class players for each week
        for x in range(1,10):
            season_stats = i[f"week_{x}"][0]
            i[f"week_{x}"][0]["Fantasy Points"] = (
            season_stats['FG 0-19'] * 3
            + season_stats['FG 20-29'] * 3
            + season_stats['FG 30-39'] * 3
            + season_stats['FG 40-49'] * 4
            + season_stats['FG 50+'] * 5
            + season_stats['PAT Made']
        )
    #Calculate fantasy football point totals for all defensive class teams
    else:
        season_stats = i["season"][0]  
        i["Fantasy Points"] = (
            + season_stats['Sack']
            + season_stats['Int'] * 2
            + season_stats['Fum Rec'] * 2
            + season_stats['TD'] * 6
            + season_stats['Safe'] * 2
            + season_stats['Blk Kick'] * 2
            + season_stats['Ret Yds'] / 15
            + season_stats['Ret TD'] * 6
            + season_stats['Pts Allow 0'] * 10
            + season_stats['Pts Allow 1-6'] * 7
            + season_stats['Pts Allow 7-13'] * 4
            +  season_stats['Pts Allow 14-20'] * 1
            + season_stats['Pts Allow 21-27'] * 0
            + season_stats['Pts Allow 28-34'] * (-1)
            + season_stats['Pts Allow 35+'] * (-4)
            + season_stats["Yds Allow 0-99"] * 10
            + season_stats["Yds Allow 100-199"] * 7
            + season_stats["Yds Allow 200-299"] * 4
            + season_stats["Yds Allow 300-399"] * 0
            + season_stats["Yds Allow 400-499"] * (-3)
            + season_stats["Yds Allow 500+"] * (-6)
            + season_stats["XPR"] * 2
        )  
        i["season"][0]["Fantasy Points"] = (
            + season_stats['Sack']
            + season_stats['Int'] * 2
            + season_stats['Fum Rec'] * 2
            + season_stats['TD'] * 6
            + season_stats['Safe'] * 2
            + season_stats['Blk Kick'] * 2
            + season_stats['Ret Yds'] / 15
            + season_stats['Ret TD'] * 6
            + season_stats['Pts Allow 0'] * 10
            + season_stats['Pts Allow 1-6'] * 7
            + season_stats['Pts Allow 7-13'] * 4
            +  season_stats['Pts Allow 14-20'] * 1
            + season_stats['Pts Allow 21-27'] * 0
            + season_stats['Pts Allow 28-34'] * (-1)
            + season_stats['Pts Allow 35+'] * (-4)
            + season_stats["Yds Allow 0-99"] * 10
            + season_stats["Yds Allow 100-199"] * 7
            + season_stats["Yds Allow 200-299"] * 4
            + season_stats["Yds Allow 300-399"] * 0
            + season_stats["Yds Allow 400-499"] * (-3)
            + season_stats["Yds Allow 500+"] * (-6)
            + season_stats["XPR"] * 2
        )
        #Calculate fantasy football point totals for all defensive class teams each week
        for x in range(1,10):
            season_stats = i[f"week_{x}"][0]
            i[f"week_{x}"][0]["Fantasy Points"] = (
                + season_stats['Sack']
            + season_stats['Int'] * 2
            + season_stats['Fum Rec'] * 2
            + season_stats['TD'] * 6
            + season_stats['Safe'] * 2
            + season_stats['Blk Kick'] * 2
            + season_stats['Ret Yds'] / 15
            + season_stats['Ret TD'] * 6
            + season_stats['Pts Allow 0'] * 10
            + season_stats['Pts Allow 1-6'] * 7
            + season_stats['Pts Allow 7-13'] * 4
            +  season_stats['Pts Allow 14-20'] * 1
            + season_stats['Pts Allow 21-27'] * 0
            + season_stats['Pts Allow 28-34'] * (-1)
            + season_stats['Pts Allow 35+'] * (-4)
            + season_stats["Yds Allow 0-99"] * 10
            + season_stats["Yds Allow 100-199"] * 7
            + season_stats["Yds Allow 200-299"] * 4
            + season_stats["Yds Allow 300-399"] * 0
            + season_stats["Yds Allow 400-499"] * (-3)
            + season_stats["Yds Allow 500+"] * (-6)
            + season_stats["XPR"] * 2
            )
            
teamPlayers

[{'player_id': 25785,
  'name': 'Russell Wilson',
  'position_type': 'O',
  'eligible_positions': ['QB'],
  'percent_owned': 16,
  'status': '',
  'season': [{'player_id': 25785,
    'name': 'Russell Wilson',
    'position_type': 'O',
    'Pass Yds': 542.0,
    'Pass TD': 3.0,
    'Int': 0.0,
    'Sack': 5.0,
    'Rush Att': 6.0,
    'Rush Yds': 10.0,
    'Rush TD': 1.0,
    'Rec': 0.0,
    'Rec Yds': 0.0,
    'Rec TD': 0.0,
    'Ret Yds': 0.0,
    'Ret TD': 0.0,
    '2-PT': 0.0,
    'Fum': 1.0,
    'Fum Lost': 1.0,
    'Fum Ret TD': 0.0,
    'Pick Six': 0.0,
    '40 Yd Comp': 2.0,
    '40 Yd Pass TD': 0.0,
    '40 Yd Rush': 0.0,
    '40 Yd Rush TD': 0.0,
    '40 Yd Rec': 0.0,
    '40 Yd Rec TD': 0.0,
    'Targets': 0.0,
    'Fantasy Points': 39.68}],
  'week_1': [{'player_id': 25785,
    'name': 'Russell Wilson',
    'position_type': 'O',
    'Pass Yds': 0.0,
    'Pass TD': 0.0,
    'Int': 0.0,
    'Sack': 0.0,
    'Rush Att': 0.0,
    'Rush Yds': 0.0,
    'Rush TD': 0.0,
    'Rec': 0

In [207]:
#save file as a json for safekeeping
import json
with open("nflData.json", 'w') as json_file:
    json.dump(teamPlayers, json_file, indent=4)
print("Data saved to output.json")

Data saved to output.json
